# Example UDF Sum-of-squares

This turtorial performs the following steps:

1. Create input table `udf_sos_in` with 1000 rows of random data for $x_1$ and $x_2$.
2. Create output table `udf_sos_out` where results will be stored.
2. Create a UDF that will calculate $y={x_1}^2+{x_2}^2 $ from the input table and save results to the output table.
4. Execute the UDF.
5. Compare the actual and expected results.

Also See:
* [Sum of Squares Tutorial](https://www.kinetica.com/docs/udf/python/examples/dist_noncuda_sum_of_squares/dist_noncuda_sum_of_squares.html)
* [Running Python UDFs](https://www.kinetica.com/docs/udf/python/running.html)
* [Python UDF API](https://www.kinetica.com/docs/udf/python/writing.html)
* [UDF Simulator](https://www.kinetica.com/docs/udf/simulating_udfs.html)


### Import dependencies

In [1]:
# Local libraries should automatically reload
%reload_ext autoreload
%autoreload 1

# to access Kinetica Jupyter I/O functions
import sys
sys.path.append('../KJIO') 

import numpy as np
import pandas as pd

%aimport kodbc_io
%aimport kapi_io

INPUT_TABLE = 'udf_example_sos_in'
OUTPUT_TABLE = 'udf_example_sos_out'
SCHEMA = 'TEST'

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Create input data table

Create a table named with `udf_sos_in` with 1000 rows of random numbers into x1 and x2 colomns. 

In [2]:
NUM_ROWS = 1000

# Create a dataframe from a dict of series. 
_input_df = pd.DataFrame({ 
    'id' : np.array(range(NUM_ROWS), dtype='int32'),
    'x1' : pd.Series(np.random.randn(NUM_ROWS)*10, dtype='float32'),
    'x2' : pd.Series(np.random.randn(NUM_ROWS)*10, dtype='float32'),
    }).set_index('id')

kapi_io.save_df(_input_df, INPUT_TABLE, SCHEMA)

Dropping table: <udf_example_sos_in>
Creating table: <udf_example_sos_in>
Column 0: <id> (long) ['primary_key']
Column 1: <x1> (float) []
Column 2: <x2> (float) []
Inserted rows into <TEST.udf_example_sos_in>: 1000


### View Input table contents

In [3]:
kodbc_io.get_df("""
select top 10 * from {}
""".format(INPUT_TABLE))

Connected to GPUdb ODBC Server (6.2.0.12.20180720232954)


,id,x1,x2
0,0,-10.361672,13.911313
1,1,-7.041492,10.108344
2,2,13.620737,-3.828699
3,3,14.187577,-7.668947
4,4,13.199041,2.597409
5,5,12.789336,-13.196888
6,9,0.255070,-14.228614
7,16,-3.547219,-5.768481
8,19,-5.373274,-10.674331
9,22,3.114088,18.321789


### Create an empty output table

In [4]:
_output_df = pd.DataFrame({ 
    'id' : pd.Series(None, dtype='int32'),
    'y' : pd.Series(None, dtype='float32'),
    }).set_index('id')

kapi_io.save_df(_output_df, OUTPUT_TABLE, SCHEMA)

Dropping table: <udf_example_sos_out>
Creating table: <udf_example_sos_out>
Column 0: <id> (long) ['primary_key']
Column 1: <y> (float) []
Inserted rows into <TEST.udf_example_sos_out>: 0


### Below is the contents of the UDF

A python file named `udf_sos_proc.py` will be saved in the current folder

In [5]:
%%writefile udf_sos_proc.py
from kinetica_proc import ProcData
proc_data = ProcData()

proc_name = proc_data.request_info['proc_name']
data_segment_id = proc_data.request_info['data_segment_id']
run_id = proc_data.request_info['run_id']
print('UDF Start: {} ({}-{})'.format(proc_name, run_id, data_segment_id))

in_table = proc_data.input_data[0]
col_in_x1 = in_table['x1']
col_in_x2 = in_table['x2']
col_in_id = in_table['id']

out_table = proc_data.output_data[0]
col_out_y = out_table['y']
col_out_id = out_table['id']

# Extend the output table by the number of record entries in the input table
out_table.size = in_table.size

# Use the first column in the output table as the output column
# Loop through all the input table columns
for i in xrange(0, in_table.size):
    col_out_y[i] = col_in_x1[i]**2 + col_in_x2[i]**2
    col_out_id[i] = col_in_id[i]

# we will get the results when the proc finishes
result_rows = str(out_table.size)
proc_data.results['result_rows'] = result_rows
proc_data.complete()

print('UDF Complete: {} rows ({}-{})'.format(result_rows, run_id, data_segment_id))

Overwriting udf_sos_proc.py


### Execute the UDF

Submit the script for execution and monitor the results.

In [6]:
%aimport kudf_io

kudf_io.create_proc(
    _proc_name='sos_proc',
    _file_paths=['./udf_sos_proc.py'])

_result = kudf_io.submit_proc(_proc_name='sos_proc', 
                    _params={},
                    _input_table_names=[INPUT_TABLE], 
                    _output_table_names=[OUTPUT_TABLE])

Reading file: udf_sos_proc.py
Creating UDF: sos_proc [./udf_sos_proc.py]
Dropping older version of proc: sos_proc 
Starting UDF: sos_proc (id=7)
   Input Tables: ['udf_example_sos_in']
   Output Tables: ['udf_example_sos_out']
[7] UDF Running... (0/2 complete) (time=0.0)
[7] UDF Running... (2/2 complete) (time=5.0)
[7] UDF finished with status: complete 
TOM 0: [complete] {'result_rows': '516'}  (time=0.6 sec)
TOM 1000: [complete] {'result_rows': '484'}  (time=0.6 sec)


{'0': {'result_rows': '516'}, '1000': {'result_rows': '484'}}

### Query the results

We should see that the `diff` column shows zero.

In [7]:
kodbc_io.get_df('''
SELECT 
    in_t.x1, 
    in_t.x2, 
    out_t.y AS actual_result, 
    FLOAT(in_t.x1 * in_t.x1 + in_t.x2 * in_t.x2) AS expected_result,
    FLOAT(in_t.x1 * in_t.x1 + in_t.x2 * in_t.x2) - out_t.y AS diff
FROM {} as out_t
INNER JOIN {} AS in_t 
    ON in_t.id = out_t.id
LIMIT 10
'''.format(OUTPUT_TABLE, INPUT_TABLE))

Connected to GPUdb ODBC Server (6.2.0.12.20180720232954)


,x1,x2,actual_result,expected_result,diff
0,-10.361672,13.911313,300.888885,300.888885,0.000000
1,-7.041492,10.108344,151.761215,151.761230,0.000015
2,13.620737,-3.828699,200.183411,200.183411,0.000000
3,14.187577,-7.668947,260.100098,260.100098,0.000000
4,13.199041,2.597409,180.961227,180.961227,0.000000
5,12.789336,-13.196888,337.724976,337.724976,0.000000
6,0.255070,-14.228614,202.518509,202.518509,0.000000
7,-3.547219,-5.768481,45.858131,45.858131,0.000000
8,-5.373274,-10.674331,142.813416,142.813416,0.000000
9,3.114088,18.321789,345.385498,345.385498,0.000000
